# Heart Disease Diagnosis

This project has been completed as a final project for *BME 440 - Introductory Python with Biomedical Applications* at Washington University in St. Louis. The project has been completed with **Arthur Li (arthurli@wustl.edu)** and **Sai Akhil Pulikam**.

The majority of the contents in this notebook have been adapted straight from the initial project proposal and the code file, but additional explanations have been added for better readability.

## Background
### Biomedical/Healthcare Need
Cardiovascular diseases are a major risk factor that compromises the health of children, men, and women in the US. Heart diseases cost the United States approximately \\$363 billion each year, and the cost is ever-growing. The average cost of most common diagnostic tests for heart diseases such as EKGs and exercise stress tests are \\$50 and \>\\$175, respectively. Sometimes, the high cost of diagnosing the disease leads to a disproportionate amount of deaths in households with lower income. Therefore, there is a need for a low-cost diagnostic product that assists patients to gauge their risk of developing heart diseases so that they can make informed decisions whether to visit a clinic to receive the actual test or not. 

(Reference: https://www.cdc.gov/heartdisease/facts.htm, https://www.choosingwisely.org/patient-resources/ekgs-and-exercise-stress-tests/)

### Project Overview
The final outcome of this project would be to create an interactive software that uses answers to a set of questions to determine whether a patient is at either a high risk or low risk of having heart diseases. Additionally, there would be a functionality to search for nearby hospitals based on the patient's provided current location. 

# Loading and Preparing Data
The 'Heart Disease Dataset' https://www.kaggle.com/datasets/johnsmith88/heart-disease-dataset by Cleveland Clinic Foundation was loaded from *Kaggle*.

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: 

Abort: 